In [4]:
import pickle
import sqlite3
import pandas as pd

In [19]:
pickle_file = 'df_topic_and_loc.pkl'  

with open(pickle_file, 'rb') as f:
    data = pickle.load(f)

data['date_published'] = data.date_published.astype(str)
data = data.drop('topic_list', axis='columns')
data.sort_values(by='date_published')

,date_published,publisher,title,text,article_url,topic,location,main_site
266,2018-10-03,Bbc,Westminster attack: The issues raised at the i...,The Westminster terror attack of 22 March 2017...,https://www.bbc.co.uk/news/uk-45719150,Children and Women's Rights | Terrorism and Ex...,NaN,www.bbc.co.uk
330,2018-10-03,Dailymail,EU considers trade sanctions on Myanmar over R...,By Robin Emmott and Philip Blenkinsop\nBRUSSEL...,https://www.dailymail.co.uk/wires/reuters/arti...,Immigration | Minorities and Human Rights | Po...,Myanmar,www.dailymail.co.uk
385,2018-10-03,Dailymail,Egypt's security forces kill 15 suspected mili...,"CAIRO, Oct 3 (Reuters) - Egyptian security for...",https://www.dailymail.co.uk/wires/reuters/arti...,Business and Economy | Politics | Minorities a...,Egypt,www.dailymail.co.uk
384,2018-10-03,Theguardian,Fethullah Gülen: shot fired at 'intruder' at T...,The Turkish cleric Fethullah Gülen at his home...,https://www.theguardian.com/us-news/2018/oct/0...,Crimes and Arrests | Politics | Children and W...,United States,www.theguardian.com
260,2018-10-03,Dailymail,Germany says shares US goals on Iran,German Foreign Minister Heiko Maas (l) met in ...,https://www.dailymail.co.uk/wires/afp/article-...,Politics,"Iran, Islamic Republic of",www.dailymail.co.uk
...,...,...,...,...,...,...,...,...
70,2019-09-28,Dailymail,US rejects Iran FM hospital visit unless Ameri...,"US says Iran's top diplomat, Javad Zarif, can ...",https://www.dailymail.co.uk/wires/afp/article-...,Immigration | Politics,"Iran, Islamic Republic of",www.dailymail.co.uk
79,2019-09-28,Express,India vs Pakistan: Trump urged to stop nuclear...,"And, responding to Mr Trump’s remark that he h...",https://www.express.co.uk/news/world/1183704/i...,Business and Economy | Politics | Terrorism an...,India,www.express.co.uk
103,2019-09-29,Dailystar,Saudi Arabia has 'thousands' of troops capture...,"Yemen’s Houthi movement, alleged to be backed ...",https://www.dailystar.co.uk/news/world-news/sa...,Business and Economy | Minorities and Human Ri...,Saudi Arabia,www.dailystar.co.uk
134,2019-09-29,Standard,Shamima Begum latest: Priti Patel says there's...,Home Secretary\nPriti Patel\nhas said there's ...,https://www.standard.co.uk/news/uk/shamima-beg...,Immigration | Children and Women's Rights | Te...,United Kingdom,www.standard.co.uk


In [89]:
data.publisher.value_counts()

publisher
Dailymail         241
Theguardian        57
Independent        41
Express            25
Bbc                23
Mirror             17
Thesun             17
Inews              14
Metro              13
Standard           12
News.Sky           12
Itv                10
Dailystar           7
Thejc               6
Christiantoday      5
Name: count, dtype: int64

In [22]:
db_path = 'cfmm.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

table_name = 'articles'
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date_published TEXT,
    publisher TEXT,
    title TEXT,
    text TEXT,
    article_url TEXT,
    topic TEXT,
    location TEXT,
    main_site TEXT
);
"""

cursor.execute(create_table_query)

# 5. Prepare an SQL INSERT query
insert_query = f"""
INSERT INTO {table_name} (
    date_published, publisher, title, text, article_url, 
    topic, location, main_site
) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""

# 6. Iterate through the DataFrame and insert each row
for _, row in data.iterrows():
    # Convert the row to a tuple of values
    values = tuple(row[col] for col in data.columns)
    cursor.execute(insert_query, values)

# 6. Commit and close the connection
conn.commit()
conn.close()


In [42]:
db_path = 'cfmm.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

sql_query = 'SELECT * FROM articles WHERE publisher in '

conn = sqlite3.connect(db_path)
pd.read_sql_query(sql_query, conn)

,id,date_published,publisher,title,text,article_url,topic,topic_list,location,main_site
0,17,2018-12-24,Dailymail,Turkey-backed fighters prepare to replace US f...,BEIRUT (AP) - Turkish-backed fighters said Mon...,https://www.dailymail.co.uk/wires/ap/article-6...,Immigration | Business and Economy | Terrorism...,None,Syrian Arab Republic,www.dailymail.co.uk
1,30,2018-12-17,News.Sky,Janbaz Tarin jailed for life for murdering ex-...,'They were let down by the authorities'\nA 21-...,https://news.sky.com/story/janbaz-tarin-man-ad...,Children and Women's Rights | Terrorism and Ex...,None,Afghanistan,news.sky.com
2,55,2018-12-28,Dailymail,Netanyahu hails 'new era' with Brazil,Netanyahu was to meet Bolsonaro for lunch in R...,https://www.dailymail.co.uk/wires/afp/article-...,Business and Economy | Terrorism and Extremism...,None,Brazil,www.dailymail.co.uk
3,63,2018-12-16,Dailymail,Iran confirms death of blogger held on securit...,"DUBAI, Dec 16 (Reuters) - Iranian authorities ...",https://www.dailymail.co.uk/wires/reuters/arti...,Immigration | Crimes and Arrests | Minorities ...,None,"Iran, Islamic Republic of",www.dailymail.co.uk
4,75,2018-12-03,Independent,The fired League Against Cruel Sports employee...,Jordi Casamitjana was sacked from The League A...,https://www.independent.co.uk/voices/vegan-fir...,Education | Business and Economy | Children an...,None,United Kingdom,www.independent.co.uk
5,83,2018-12-24,Dailymail,The Latest: 29 killed in attack in Afghanistan...,"KABUL, Afghanistan (AP) - The Latest on the at...",https://www.dailymail.co.uk/wires/ap/article-6...,Terrorism and Extremism,None,Afghanistan,www.dailymail.co.uk
6,96,2018-12-31,Dailymail,Bahrain top court upholds sentence against act...,"By Aziz El Yaakoubi\nDUBAI, Dec 31 (Reuters) -...",https://www.dailymail.co.uk/wires/reuters/arti...,Crimes and Arrests | Politics | Immigration | ...,None,Bahrain,www.dailymail.co.uk
7,117,2018-12-16,Dailymail,Israeli PM's son gets temporary ban on Facebook,JERUSALEM (AP) - Facebook has blocked Israeli ...,https://www.dailymail.co.uk/wires/ap/article-6...,Terrorism and Extremism | Politics | Hate Spee...,None,Israel,www.dailymail.co.uk
8,121,2018-12-22,Theguardian,Al-Shabaab car bomb kills at least 16 in Mogad...,Somali women walk past wreckage of the car bom...,https://www.theguardian.com/world/2018/dec/22/...,Children and Women's Rights | Crimes and Arres...,None,Ethiopia,www.theguardian.com
9,148,2018-12-30,Dailymail,Iraq hints at bigger role in Syria after U.S. ...,"BAGHDAD, Dec 30 (Reuters) - Iraq's prime minis...",https://www.dailymail.co.uk/wires/reuters/arti...,Business and Economy | Politics | Immigration ...,None,Syrian Arab Republic,www.dailymail.co.uk


In [35]:
[i for i in cursor.execute(sql_query)][0]

('2018-10-03', '2019-09-30')

In [36]:
from datetime import datetime

In [47]:
datetime.strptime('2018-10-03', '%Y-%m-%d').strftime('%Y-%m-%d')

'2018-10-03'

In [86]:
def build_query(selected_publisher, start_date, end_date, compared_publishers, topics):
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')
    sql = f"SELECT * FROM articles WHERE (date_published >= '{start_date}' AND date_published <= '{end_date}')"

    publishers = [selected_publisher] + compared_publishers
    if len(publishers) > 0:
        publisher_sql = ' AND (' + ' OR '.join([f"publisher LIKE '%{i}%'" for i in publishers]) + ')'
        sql += publisher_sql

    if len(topics) > 0:
        topic_sql = ' AND (' + ' OR '.join([f"topic LIKE '%{i}%'" for i in topics]) + ')'
        sql += topic_sql
    
    return sql
    

In [87]:
sql_query = build_query('Bbc',
            datetime.strptime('2018-10-03', '%Y-%m-%d'),
            datetime.strptime('2019-01-01', '%Y-%m-%d'),
            # ['Dailymail', 'Express', 'Independent'],
            [],
            ['Immigration'])

sql_query

"SELECT * FROM articles WHERE (date_published >= '2018-10-03' AND date_published <= '2019-01-01') AND (publisher LIKE '%Bbc%') AND (topic LIKE '%Immigration%')"

In [88]:
pd.read_sql_query(sql_query, conn)

,id,date_published,publisher,title,text,article_url,topic,topic_list,location,main_site


In [70]:
pd.read_sql_query("SELECT * FROM articles WHERE (date_published >= '2018-10-03' AND date_published <= '2019-01-01')", conn)

,id,date_published,publisher,title,text,article_url,topic,topic_list,location,main_site
0,6,2018-11-18,Mirror,ISIS bride sisters held in Syria are part of U...,Two sisters who are both brides of men who die...,https://www.mirror.co.uk/news/uk-news/isis-bri...,Children and Women's Rights | Crimes and Arres...,None,Syrian Arab Republic,www.mirror.co.uk
1,17,2018-12-24,Dailymail,Turkey-backed fighters prepare to replace US f...,BEIRUT (AP) - Turkish-backed fighters said Mon...,https://www.dailymail.co.uk/wires/ap/article-6...,Immigration | Business and Economy | Terrorism...,None,Syrian Arab Republic,www.dailymail.co.uk
2,24,2018-10-19,Inews,Are you smarter than your kid? Take this quiz ...,Take the quiz:\nPick the correct definition of...,https://inews.co.uk/light-relief/quizzes/are-y...,Children and Women's Rights | Business and Eco...,None,United States,inews.co.uk
3,29,2018-10-29,Theguardian,Black Earth Rising recap: the finale – it has ...,Michaela Coel gives a magnificent performance ...,https://www.theguardian.com/tv-and-radio/2018/...,Business and Economy | Crimes and Arrests | Po...,None,Rwanda,www.theguardian.com
4,30,2018-12-17,News.Sky,Janbaz Tarin jailed for life for murdering ex-...,'They were let down by the authorities'\nA 21-...,https://news.sky.com/story/janbaz-tarin-man-ad...,Children and Women's Rights | Terrorism and Ex...,None,Afghanistan,news.sky.com
...,...,...,...,...,...,...,...,...,...,...
117,470,2018-10-05,Dailymail,Pakistan opposition leader Shahbaz Sharif arre...,Former Pakistani Prime Minister Nawaz Sharif a...,https://www.dailymail.co.uk/wires/afp/article-...,Crimes and Arrests | Politics | Immigration | ...,None,Pakistan,www.dailymail.co.uk
118,474,2018-12-22,Independent,The A-Z of Believing: S is for Sex | The Indep...,Song of Songs 7: 9-10,https://www.independent.co.uk/life-style/sex-s...,,None,None,www.independent.co.uk
119,477,2018-12-13,Independent,Government urged to adopt new definition of Is...,The government has been urged by more than 50 ...,https://www.independent.co.uk/news/uk/politics...,Children and Women's Rights | Politics | Hate ...,None,United Kingdom,www.independent.co.uk
120,481,2018-11-09,Dailymail,4 car bombs explode by hotel in Somalia's capi...,"MOGADISHU, Somalia (AP) - Four car bombs by Is...",https://www.dailymail.co.uk/wires/ap/article-6...,Terrorism and Extremism,None,Somalia,www.dailymail.co.uk
